In [1]:
import os 
!pwd
os.chdir("../")
!pwd

/Users/debarchan/PycharmProjects/Adenocarcinoma-Detection/research
/Users/debarchan/PycharmProjects/Adenocarcinoma-Detection


In [2]:
import tensorflow as tf 
import dagshub
import mlflow

2024-06-08 22:27:35.818295: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    validation_data: Path
    testing_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [5]:
from carcinomaDetection.constants import *
from carcinomaDetection.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        
        evaluation_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Data/train",
            validation_data="artifacts/data_ingestion/Data/valid",
            testing_data="artifacts/data_ingestion/Data/test",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return evaluation_config

In [7]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [8]:
class Evaluation:
  def __init__(self, config: EvaluationConfig):
    self.config = config

  def _valid_generator(self):

    datagenerator_kwargs = dict(
        rescale = 1./255,
    )

    dataflow_kwargs = dict(
        target_size=self.config.params_image_size[:-1],
        batch_size=self.config.params_batch_size,
        interpolation="bilinear"
    )

    valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        **datagenerator_kwargs
    )

    self.valid_generator = valid_datagenerator.flow_from_directory(
        directory=self.config.testing_data,
        shuffle=False,
        **dataflow_kwargs
    )

  @staticmethod
  def load_model(path: Path) -> tf.keras.Model:
    return tf.keras.models.load_model(path)

  def evaluation(self):
    self.model = self.load_model(self.config.path_of_model)
    self._valid_generator()
    self.score = model.evaluate(self.valid_generator)

  def save_score(self):
    scores = {"loss": self.score[0], "accuracy": self.score[1]}
    save_json(path=Path("scores.json"), data=scores)

  def log_into_mlflow(self):
    dagshub.init(repo_owner='quantumsleeper', repo_name='Adenocarcinoma-Detection', mlflow=True)

    with mlflow.start_run():
      mlflow.log_params(self.config.all_params)
      mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
      
      mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16")


In [9]:
try:
  config = ConfigurationManager()
  eval_config = config.get_evaluation_config()
  evaluation = Evaluation(eval_config)
  evaluation.evaluation()
  evaluation.save_score()
  evaluation.log_into_mlflow()

except Exception as e:
  raise e

[2024-06-08 22:27:54,009: INFO: common: yaml file: config/config.yaml loaded successfully!]
[2024-06-08 22:27:54,014: INFO: common: yaml file: params.yaml loaded successfully!]
[2024-06-08 22:27:54,017: INFO: common: created directory at: artifacts]
Found 315 images belonging to 4 classes.


2024-06-08 22:27:55.054182: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


20/20 [==============================] - 50s 2s/step - loss: 1.6549 - accuracy: 0.3111
[2024-06-08 22:28:45,220: INFO: common: json file saved at: scores.json]


Initialized MLflow to track repo "quantumsleeper/Adenocarcinoma-Detection"

[2024-06-08 22:28:48,032: INFO: helpers: Initialized MLflow to track repo "quantumsleeper/Adenocarcinoma-Detection"]


Repository quantumsleeper/Adenocarcinoma-Detection initialized!

[2024-06-08 22:28:48,035: INFO: helpers: Repository quantumsleeper/Adenocarcinoma-Detection initialized!]


2024/06/08 22:28:49 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-06-08 22:28:51,547: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /var/folders/f1/0g56z6153rv3gh97q2kwzhmh0000gn/T/tmpesqwi1em/model/data/model/assets
[2024-06-08 22:28:52,275: INFO: builder_impl: Assets written to: /var/folders/f1/0g56z6153rv3gh97q2kwzhmh0000gn/T/tmpesqwi1em/model/data/model/assets]


/opt/anaconda3/envs/carcinoma/lib/python3.8/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16' already exists. Creating a new version of this model...
2024/06/08 22:29:26 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16, version 2
Created version '2' of model 'VGG16'.
